In [2]:
#import class_database 
#import pymysql
#from pymysql.constants import CLIENT
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import dendrogram, to_tree, ClusterNode
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib as mpl
import datetime
import os
from itertools import accumulate
from sklearn.metrics.pairwise import pairwise_distances, euclidean_distances
from scipy.stats import entropy
from math import floor
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from patsy import dmatrices
from tqdm.auto import tqdm
#from Levenshtein import distance as lev
import re

/home/alberto/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
_HOME_ = "/home/alberto"

In [5]:
def remove_additional_white_spaces(string): 
    return " ".join(string.strip().split())

In [6]:
con = pymysql.connect(host="localhost",
                          user="PhD_alberto",
                          password="MySQL@4869",
                          autocommit=True,
                          client_flag=CLIENT.MULTI_STATEMENTS)


database = class_database.Database(con, "MSFD_ARPA_20152017_MOD1")

database.switch_to_db()

NameError: name 'pymysql' is not defined

In [ ]:
df_lat_long_stations = database.extract_data("Stations", 
                                             ["id", "Latitude", "Longitude", "Closest_coast"], 
                                             join_type_table_names={"INNER" : ["Regions"]},
                                             join_type_select={"INNER" : [["Region"]]},
                                             join_type_on={"INNER" : [{"Regions.id" : "Stations.Reg_id"}]}
                                             )
phyto_points = df_lat_long_stations[["id", "Longitude", "Latitude"]].dropna().set_index("id")

In [ ]:
phyto_abund_raw = pd.read_csv(_HOME_ + "/PHD/MSFD/Data/Modulo1/phyto_abund_raw.csv")
phyto_abund_raw.rename(columns={"NationalStationID" : "id", "Gruppo" : "Group", "Time" : "time"}, inplace=True)
phyto_abund_raw["id"]  = phyto_abund_raw["id"].apply(lambda x: x.split(".")[0])
dates = [
    datetime.date(year=year, month=month, day=day) 
    for year, month, day in phyto_abund_raw.loc[:, ["Year", "Month", "Day"]].to_numpy(np.int16)    
]
phyto_abund_raw.drop(columns = ["Unnamed: 0", "Autore", "Year", "Month", "Day"], inplace=True)
phyto_abund_raw["Day"] = dates
phyto_abund_raw.loc[phyto_abund_raw["id"] == '0004-M000200_TR03', "id"] = '0004-MS00200_TR03'
phyto_abund_raw.loc[phyto_abund_raw["id"] == '0004-M000200_TR06', "id"] = '0004-MS00200_TR06'
phyto_abund_raw.loc[phyto_abund_raw["id"] == '0004-M000200_TR12', "id"] = '0004-MS00200_TR12'

/tmp/ipykernel_8994/977035052.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  phyto_abund_raw = pd.read_csv("/mnt/d/PHD/MSFD/Data/Modulo1/phyto_abund_raw.csv")


In [ ]:
phyto_abund_raw.dropna(subset=["Num_cell_l"], inplace=True)
phyto_abund_raw.loc[84343,"Group"] = "Dinoflagellati"
phyto_abund_raw.loc[124333, "Group"] = "Diatomee"

In [ ]:
phyto_list_excel = pd.read_excel(_HOME_ + "/PHD/MSFD/Data/Modulo1/2019_01_mod_1_FITO.xls", "Lista_specie_fitoplancton")
phyto_list_excel = phyto_list_excel["TAXON"]
phyto_list_excel = phyto_list_excel.apply(remove_additional_white_spaces)

In [ ]:
phyto_abund_raw = phyto_abund_raw.merge(df_lat_long_stations[["id", "Region", "Longitude", "Latitude", "Closest_coast"]], how = "left", left_on="id", right_on="id")

In [ ]:
phyto_abund_raw["Taxon"] = np.where(phyto_abund_raw["Taxon"].isna(), phyto_abund_raw["NuovoTaxon"], phyto_abund_raw["Taxon"])
phyto_abund_raw.drop(columns = ["NuovoTaxon"], inplace=True)
phyto_abund_raw.loc[~phyto_abund_raw["Taxon"].isna(), "Taxon"] = phyto_abund_raw.loc[~phyto_abund_raw["Taxon"].isna(), "Taxon"].apply(remove_additional_white_spaces)
phyto_abund_raw.loc[phyto_abund_raw["Taxon"].isna(), "Taxon"] = "Altro fitoplancton"

In [ ]:
phyto_abund_raw["Taxon"] = phyto_abund_raw["Taxon"].apply(str.capitalize)

Errors: <br> 
Diatomee as genus <br>
Gymnodiniaceae is a Family <br>
Prasinophyceae is a class <br>
Cisti is Cisticula? <br>
Brachidinium is CORRECT Brachydinium IS NOT CORRECT <br>
Oscillatoriales is an order but Oscillatoria is a genus<br>
Cyanophyceae is a class <br>
Gymnodiniales is an order <br>
Bacillariales is an order <br>
Eutreptiaceae is a family <br>
Naviculaceae is a family <br>
Cianophyceae : Cyanophyceae but is a Class <br>
Pedinellaceae is a family <br>
Bacillariophyta is a phylum <br>
Peridiniales is an order <br>
Flagellati is a group <br>
Flagellato is Flagellati? <br>
Coccolitophyceae : Coccolithophyceae but is a class <br>
Coccolithopyceae : Coccolithophyceae but is a class <br>
Chriptophyceae : Cryptophyceae but is a class <br>
Pseudonitzschia : Pseudo-nitzschia the first one is better <br>
Gonyaulacaceae is a family <br>
Chlorophyta is a phylum <br>
Dinoflagellata is a infraphylum <br>
Prymnesiales is a order <br>
Coccolithphyceae : Coccolithophyceae but is a class <br>
Pedinellales is an order <br>
Pseudo-nitzscha : Pseudo-nitzschia but Pseudonitzschia is better <br>
Bacillariopyta : Bacillariophyta but is a phylum <br>
Cianobatteri is a phylum <br>
Prococystis : Podocystis or Photocystis <br>
Haptophyta is a phylum <br>
Chroococcales is an order <br>


In [ ]:
corrected_genus = {"Cf.." : "Cf.", 
 "Phalachroma" : "Phalacroma", 
 "Protoperidium": "Protoperidinium",
 "Nefroselmis" : "Nephroselmis",
 "Minuscola" : "Minuscula",
 "Tecati" : "Tecatia", 
 "Heteroleiblenia" : "Heteroleibleinia", 
 "Haloshpaera" : "Halosphaera", 
 "Distephanus" : "Distephanopsis", 
 "Talassionema" : "Thalassionema", 
 "Hetrocapsa" : "Heterocapsa", 
 "Pyramonas" : "Pyramimonas", 
 "Nitzchia" : "Nitzschia", 
 "Chaetocheros" : "Chaetoceros", 
 "Rhodomonascf." : "Rhodomonas cf.", 
 "Syrachosphaera" : "Syracosphaera", 
 "Indeterminat" : "Indeterminato", 
 "Gunardia" : "Guinardia", 
 "Oxitoxum" : "Oxytoxum", 
 "Altro_fitoplanctonerminati" : "Altro_fitoplancton", 
 "Mesoporus" : "Mesoporos", 
 "Schuetiella" : "Schuettiella", 
 "Chatoceros" : "Chaetoceros", 
 "Dicosphaera" : "Discosphaera", 
 "Prorocenrum" : "Prorocentrum", 
 "Chrysocromulina" : "Chrysochromulina", 
 "Dictyoca" : "Dictyocha", 
 "Mesoporo" : "Mesoporos", 
 "Octastis": "Octactis", 
 "Ceratum" : "Ceratium", 
 "Schripsiella" : "Scrippsiella", 
 "Ceratulina" : "Cerataulina", 
 "Guanardia" : "Guinardia", 
 "Leptocilindrus" : "Leptocylindrus", 
 "Cilindrotheca" : "Cylindrotheca", 
 "Hemialus" : "Hemiaulus", 
 "Thalassionemasp." : "Thalassionema", 
 "Asterinellopsis" : "Asterionellopsis", 
 "Dactysolen" : "Dactyliosolen",
 "Dactylosolen" : "Dactyliosolen",
 "Psudosolenia" : "Pseudosolenia", 
 "Amphididium" : "Amphidinium", 
 "Lessar" : "Lessardia",
 "Oxyoxum" : "Oxytoxum", 
 "Crysophyceae" : "Chrysophyceae", 
 "Hasleasp" : "Haslea sp.", 
 "Gyrodinum" : "Gyrodinium", 
 "Gymnodium" : "Gymnodinium", 
 "Mesopros" : "Mesoporos", 
 "Hemialius" : "Hemiaulus", 
 "Gonylaux" : "Gonyaulax", 
 "Proprocentrum" : "Prorocentrum", 
 "Protopteridinium" : "Protoperidinium", 
 "Kofodinium" : "Kofoidinium",
 "Protopterodinium" : "Protoperidinium", 
 "Chaetocerus" : "Chaetoceros", 
 "Rhabdosphera" : "Rhabdosphaera", 
 "Girosigma" : "Gyrosigma", 
 "Protoperidinuum" : "Protoperidinium", 
 "Alexandrinum" : "Alexandrium", 
 "Brachydinium" : "Brachidinium", 
 "Pseudo-nitzschia" : "Pseudonitzschia"
 }

In [ ]:
taxa = phyto_abund_raw["Taxon"].to_numpy()

for i, taxon in enumerate(taxa): 
    try: 
        first, second = taxon.split(" ", maxsplit = 1)
        if first in corrected_genus.keys():
            first = corrected_genus[first]
        taxa[i] = first + " " + second
    except: 
        if taxon == "Hasleasp": 
            taxa[i] = taxon.replace("Hasleasp", "Haslea sp.")
        if taxon == "Thalassionemasp.":
            taxa[i] = taxon.replace("Thalassionemasp.", "Thalassionema sp.")

In [ ]:
for i, taxon in enumerate(taxa):
    taxon = re.sub("(sp\.p\.+|sp\.p\.+|spp\.+|sp\.p\.\.|spp\.\.|sp\.p|spp|sp\. p\.)", "spp.", taxon)
    taxon = re.sub("(sp$|sp.2)", "sp.", taxon)
    taxon = re.sub('(?<=[.,])(?=[^\s])', ' ', taxon)
    taxon = re.sub("indet\.*", "indet.", taxon)
    taxon = re.sub("cf\.*", "cf.", taxon)
    taxon = re.sub("(Pseudo-nitzschia|Pseudo nitzschia)", "Pseudonitzschia", taxon)
    taxa[i] = taxon

In [ ]:
#correction to my dataset
match_taxa = {
    "Gunardia striata" : "Guinardia striata", 
    "Guanardia striata" : "Guinardia striata", 
    "Gymnodinium wulfii" : "Gymnodinium wulffii",
    "Altro fitoplancton" : "Other phytoplankton", 
    "Gyrodinium fusiform" : "Gyrodinium fusiforme",
    "Gyrodinium fusiformis." : "Gyrodinium fusiforme",
    "Altro fitoplancton indet." : "Other phytoplankton",
    "Hemialius hauckii" : "Hemiaulus hauckii", 
    "Hemiaulus haukii" : "Hemiaulus hauckii",
    "Hemiaulus senensis" : "Hemiaulus sinensis",
    "Hemiaulus seniensis" :  "Hemiaulus sinensis",
    "Hemiaulus siniensis" : "Hemiaulus sinensis",
    "Heterocapsa rotondata" : "Heterocapsa rotundata",
    "Karenia papillionacea" : "Karenia papilionacea",
    "Karenia papillonacea" : "Karenia papilionacea", 
    "Kofodinium sp." : "Kofoidinium sp.",
    "Lauderia annullata" : "Lauderia annulata",
    "Leptocylindrus cf. . convexus" : "Leptocylindrus cf. convexus", 
    "Leptocylindrus miimus" : "Leptocylindrus minimus",
    "Leptocylindrus minimum" : "Leptocylindrus minimus",
    "Mesoporus adriaticus" : "Mesoporos adriaticus",
    "Mesoporus spp." : "Mesoporos spp.",
    "Mesopros sp." : "Mesoporos sp.",
    "Mesoporo" : "Mesoporo sp.",
    "Minuscola bipes" : "Minuscula bipes",
    "Navicula" : "Navicula spp.",
    "Nitzchia sp." : "Nitzschia sp.",
    "Prorocentrum compressus" : "Prorocentrum compressum",
    "Prorocentrum sp. inf 20 µm" : "Prorocentrum sp. (ø inf. 20µm)",
    "Prorocentrum sp. sup 20 µm" : "Prorocentrum sp. (ø sup. 20µm)",
    "Prorocentrum spp. sup 20 µm" : "Prorocentrum spp. (ø sup. 20µm)",
    "Group" : "Other phytoplankton",
    "Protoperidinium inf 20 µm" : "Protoperidinium indet. (ø inf. 20µm)",
    "Protoperidinium o pentapharsodinium" : "Other phytoplankton",
    "Protopteridinium steini" : "Protoperidinium steinii",
    "Protopterodinium steini" : "Protoperidinium steinii",
    "Pseudosolenia calcar avis" : "Pseudosolenia calcar-avis",
    "Psudosolenia calcar-avis" : "Pseudosolenia calcar-avis",
    "Rhizosolenia calcar avis" : "Rhizosolenia calcar-avis",
    "Rhizosolenia clevei var communis" : "Rhizosolenia clevei var. communis",
    "Thalassionema fraunfeldii" : "Thalassionema frauenfeldii",
    "Thalassionema nitzschiodies" : "Thalassionema nitzschioides",
    "Thalassionema nitzschoides" : "Thalassionema nitzschioides",
    "Syracosphaera pulcra" : "Syracosphaera pulchra",
    "Indeterminat inon flagellati sup 20 µm" : "Other phytoplankton (ø sup. 20µm)", 
    "Altro fitoplancton indet. (ø inf. 20µm)" : "Other phytoplankton (ø inf. 20µm)",
    "Altro fitoplancton indet. (ø sup. 20µm)" : "Other phytoplankton (ø sup. 20µm)", 
    "Altro_fitoplanctonerminati inf 20 µm" : "Other phytoplankton (ø sup. 20µm)", 
    "Altro_fitoplanctonerminati non flagellati sup 20 µm" : "Other phytoplankton (ø sup. 20µm)", 
    "Asteromphaluscf. parvulus" : "Asteromphalus cf. parvulus",
    "Ceratium fusum" : "Ceratium fusus",
    "Asteromphaluscf. sarcophagus" : "Asteromphalus cf. sarcophagus",
    "Altro_fitoplancton inf 20 µm" : "Other phytoplankton (ø inf. 20µm)", 
    "Altro_fitoplancton non flagellati sup 20 µm" : "Other phytoplankton (ø sup. 20µm)", 
    "Bacillariophyceae" : "Bacillariophyceae indet.", 
    "Bacillariophyceae < 20" : "Bacillariophyceae indet. (ø inf. 20µm)", 
    "Bacillariophyceae indet. < 20" : "Bacillariophyceae indet. (ø inf. 20µm)",
    "Bacillariophyceae indet. < 20um" : "Bacillariophyceae indet. (ø inf. 20µm)",
    "Bacillariophyceae inf 20 micron" : "Bacillariophyceae indet. (ø inf. 20µm)",
    "Bacillariophyceae sup 20 micron" : "Bacillariophyceae indet. (ø sup. 20µm)", 
    "Bacillariophyta" : "Bacillariophyta indet.",
    "Bacillariopyta inf 20 micron" : "Bacillariophyta indet. (ø inf. 20µm)",
    "Bacillariophyta inf 20 micron" : "Bacillariophyta indet. (ø inf. 20µm)", 
    "Bacillariophyta inf 20 µm" : "Bacillariophyta indet. (ø inf. 20µm)", 
    "Bacillariophyta inf. 20 micron" : "Bacillariophyta indet. (ø inf. 20µm)",
    "Bacillariophyta inf. 20 µm" : "Bacillariophyta indet. (ø inf. 20µm)",
    "Bacillariophyta sup 20 micon" : "Bacillariophyta indet. (ø sup. 20µm)",
    "Bacillariophyta sup 20 micron" : "Bacillariophyta indet. (ø sup. 20µm)",
    "Bacillariophyta sup 20 µm" : "Bacillariophyta indet. (ø sup. 20µm)",
    "Bacillariophyta sup. 20 micron" : "Bacillariophyta indet. (ø sup. 20µm)",
    "Bacillariophyta sup. 20 µm": "Bacillariophyta indet. (ø sup. 20µm)",
    "Bacillariophyta indet. (ø sup. 20µm)" :"Bacillariophyta indet. (ø sup. 20µm)", 
    "Cf. . amphidinium spp." : "Cf. amphidinium spp.",
    "Chaetoceros anastamousus" : "Chaetoceros anastomosans",
    "Chaetoceros anastemousus" : "Chaetoceros anastomosans",
    "Chaetoceros anastomonas" : "Chaetoceros anastomosans",
    "Chaetoceros anastomosous" : "Chaetoceros anastomosans",
    "Chaetoceros anastomosus" : "Chaetoceros anastomosans",
    "Chaetoceros anastomousus" : "Chaetoceros anastomosans",
    "Chaetoceros anastoumosus" : "Chaetoceros anastomosans", 
    "Chlorodendrophyceae" : "Chlorodendrophyceae indet.", 
    "Chlorophyceae" : "Chlorophyceae indet.", 
    "Chlorophyta" : "Chlorophyta indet.", 
    "Chriptophyceae" : "Cryptophyceae indet.", 
    "Chroococcales" : "Chroococcales indet.", 
    "Chrysochromulina" : "Chrysochromulina sp.", 
    "Chrysocromulina" : "Chrysochromulina sp.", 
    "Chrysophyceae" : "Chrysophyceae indet.",
    "Chrysophyceae indet" : "Chrysophyceae indet.", 
    "Cianobatteri filamentosi" : "Cyanobacteria",
    "Cianobatteri non filamentosi" : "Cyanobacteria", 
    "Cianophyceae indet" : "Cyanophyceae indet.", 
    "Cisti di protoperidinium sp." : "Protoperidinium sp.", 
    "Coccolithophyceae" : "Coccolithophyceae indet.",
    "Coccolithophyceae indet. < 20" : "Coccolithophyceae indet. (ø inf. 20µm)",
    "Coccolithopyceae" : "Coccolithophyceae indet.",
    "Coccolithphyceae" : "Coccolithophyceae indet.",
    "Coccolitophyceae" : "Coccolithophyceae indet.",
    "Coccolitophyceae indet." : "Coccolithophyceae indet.",
    "Coccolitophyceae sup 20 µm" : "Coccolithophyceae indet. (ø sup. 20µm)",
    "Coscinodiscophyceae" : "Coscinodiscophyceae indet.", 
    "Cryptophyceae" : "Cryptophyceae indet.",
    "Cyanophyceae" : "Cyanophyceae indet.",
    "Cyanophyceae indet" : "Cyanophyceae indet.",
    "Cyanophyceae indet filamenti" : "Cyanophyceae indet.", 
    "Diatoma" : "Diatoma sp.",
    "Diatoma s. p." : "Diatoma sp.",
    "Diatoma s. p. p." : "Diatoma spp.",
    "Diatomee indet" : "Diatomee indet.",
    "Dictyochophyceae" : "Dictyochophyceae indet.", 
    "Dinoflagellata" : "Dinoflagellata indet.",
    "Dinoflagellati" : "Dinoflagellata indet.",
    "Dinoflagellati indet." : "Dinoflagellata indet.",
    "Dinoflagellati indet. (ø inf. 20µm)" : "Dinoflagellata indet. (Ø inf. 20µm)",
    "Dinoflagellati indet. (ø sup. 20µm)" : "Dinoflagellata indet. (Ø sup. 20µm)",
    "Dinophyceae" : "Dinophyceae indet.",
    "Dinophyceae indet (ø sup. 20µm)" : "Dinophyceae indet. (ø sup. 20µm)",
    "Dinophyceae inf 20 micron" : "Dinophyceae indet. (ø inf. 20µm)",
    "Dinophyceae inferiori 20 micron" : "Dinophyceae indet. (ø inf. 20µm)",
    "Dinophyceae sup 20 micron" : "Dinophyceae indet. (ø sup. 20µm)",
    "Dinophyceae sup 20 µm" : "Dinophyceae indet. (ø sup. 20µm)",
    "Dinophyceae sup. 20 micron" : "Dinophyceae indet. (ø sup. 20µm)",
    "Dinophyceae superiori 20 micron" : "Dinophyceae indet. (ø sup. 20µm)", 
    "Euglenophyceae" : "Euglenophyceae indet.",
    "Euglenophyceae indet. < 20 µm" : "Euglenophyceae indet. (ø inf. 20µm)",
    "Euglenophyceae indet. > 20 µm" : "Euglenophyceae indet. (ø sup. 20µm)",
    "Eutreptiaceae" : "Eutreptiaceae indet.",
    "Eutreptiaceae indet" : "Eutreptiaceae indet.",
    "Eutreptiella" : "Eutreptiella sp.",
    "Fitoplancton indet." : "Other phytoplankton",
    "Flagellati inf 20 micron" : "Flagellates indet. (ø inf. 20µm)",
    "Flagellati inf 20 µm" : "Flagellates indet. (ø inf. 20µm)",
    "Flagellati inf a 20 micron" : "Flagellates indet. (ø inf. 20µm)",
    "Flagellati inf a 20 µm" : "Flagellates indet. (ø inf. 20µm)",
    "Flagellato inf 20 µm" : "Flagellates indet. (ø inf. 20µm)",
    "Gymnodiniaceae" : "Gymnodiniaceae indet.", 
    "Gymnodiniaceae inf 20 micron" : "Gymnodiniaceae indet. (ø inf. 20µm)",
    "Gymnodiniaceae inf 20 µm" : "Gymnodiniaceae indet. (ø inf. 20µm)",
    "Gymnodiniaceae sup 20 micron" : "Gymnodiniaceae indet. (ø sup. 20µm)",
    "Gymnodiniaceae sup 20 µm" : "Gymnodiniaceae indet. (ø sup. 20µm)",
    "Gymnodiniales" : "Gymnodiniales indet.",
    "Gymnodiniales (ø inf. 20µm)" : "Gymnodiniales indet. (ø inf. 20µm)",
    "Gymnodiniales inf 20 micron" : "Gymnodiniales indet. (ø inf. 20µm)",
    "Gymnodiniales inf 20 µm" : "Gymnodiniales indet. (ø inf. 20µm)",
    "Gymnodiniales sup 20 micron" : "Gymnodiniales indet. (ø sup. 20µm)",
    "Gymnodiniales sup 20 µm" : "Gymnodiniales indet. (ø sup. 20µm)",
    "Haptophyta" : "Haptophyta indet.", 
    "Ind. sup 20 micron" : "Other phytoplankton",
    "Indet" : "Other phytoplankton",
    "Indeterminati" : "Other phytoplankton",
    "Indeterminati forma siluro sup. 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
    "Indeterminati inf 20 micron" : "Other phytoplankton (Ø inf. 20µm)",
    "Indeterminati inf 20 µm" : "Other phytoplankton (Ø inf. 20µm)",
    "Indeterminati inf. 20 micron" : "Other phytoplankton (Ø inf. 20µm)",
    "Indeterminati non flag inf. 20 micron" : "Other phytoplankton (Ø inf. 20µm)",
    "Indeterminati non flag inf. 20 µm" : "Other phytoplankton (Ø inf. 20µm)",
    "Indeterminati non flag sup. 20 micron" : "Other phytoplankton (Ø sup. 20µm)",
    "Indeterminati non flag sup. 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
    "Indeterminati non flagellati" : "Other phytoplankton",
    "Indeterminati non flagellati inf 20 micron" : "Other phytoplankton (Ø inf. 20µm)",
    "Indeterminati non flagellati inf 20 µm" : "Other phytoplankton (Ø inf. 20µm)",
    "Indeterminati non flagellati sup 20 micron" : "Other phytoplankton (Ø sup. 20µm)",
    "Indeterminati non flagellati sup 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
    "Indeterminati sup 20 micron" : "Other phytoplankton (Ø sup. 20µm)",
    "Indeterminati sup 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
    "Indeterminati sup a 20 micron" : "Other phytoplankton (Ø sup. 20µm)",
    "Indeterminati sup. 20 micron" : "Other phytoplankton (Ø sup. 20µm)",
    "Indeterminato inon flagellati sup 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
    "Indeterminato non flagellato inf 20 µm" : "Other phytoplankton (Ø inf. 20µm)",
    "Indeterminato non flagellato sup 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
    "Indeterminato sup 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
    "Lessardia elongata > 20 um" : "Lessardia elongata",
    "Navicula sp. inf 20 µm" : "Navicula sp. (Ø inf. 20µm)",
    "Navicula sp. sup 20 µm" : "Navicula sp. (Ø sup. 20µm)",
    "Navicula spp. sup 20 micron" : "Navicula spp. (Ø sup. 20µm)",
    "Navicula spp. sup. 20 micron" : "Navicula spp. (Ø sup. 20µm)",
    "Naviculaceae indet" : "Naviculaceae indet.",
    "Octactis octonaria v. pulchra" : "Octactis octonaria var. pulchra", 
    "Oscillatoriales" : "Oscillatoriales indet.",
    "Oscillatoriales ind." : "Oscillatoriales indet.",
    "Oxytoxum variabile < 20" : "Oxytoxum variabile",
    "Oxytoxum variabile < 20 µm" : "Oxytoxum variabile",
    "Oxytoxum variabile > 20" : "Oxytoxum variabile",
    "Oxytoxum variabile > 20 µm" : "Oxytoxum variabile",
    "Pedinellaceae" : "Pedinellaceae indet.",
    "Pedinellaceae indet" : "Pedinellaceae indet.",
    "Pedinellales" : "Pedinellales indet.", 
    "Peridiniales" : "Peridiniales indet.",
    "Peridiniales inf 20 micron" : "Peridiniales indet. (Ø inf. 20µm)",
    "Peridiniales spp." : "Peridiniales indet.",
    "Peridiniales sup. 20 micron" : "Peridiniales indet. (Ø sup. 20µm)", 
    "Plagioselmis" : "Plagioselmis sp.", 
    "Prasinophyceae" : "Prasinophyceae indet.",
    "Prasinophyceae indet" : "Prasinophyceae indet.",
    "Prymnesiales" : "Prymnesium indet.",
    "Prymnesiophyceae" : "Coccolithophyceae indet.",
    "Prymnesiophyceae indet." : "Coccolithophyceae indet.",
    "Pseudo nitzschia cf. fraudolenta" : "Pseudonitzschia cf. fraudolenta",
    "Pseudo nitzschia spp." : "Pseudonitzschia spp.",
    "Pseudo nitzschia subfraudolenta" : "Pseudonitzschia subfraudolenta",
    "Pseudo-nitzscha spp." : "Pseudonitzschia spp.", 
    "Pseudonitzschia group spp." : "Pseudonitzschia spp.", 
    "Pseudonitzschia pseudelicatissima" : "Pseudonitzschia pseudodelicatissima",
    "Pseudonitzschia pseudodelicatissima group" : "Pseudonitzschia pseudodelicatissima",
    "Pseudonitzschia s." : "Pseudonitzschia sp.", 
    "Pseudosolenia calcar- avis" : "Pseudosolenia calcar-avis",
    "Pyramimonadophyceae" : "Pyramimonadophyceae indet.",
    "Raphidophyceae" : "Raphidophyceae indet.",
    "Raphidophyceae indt." : "Raphidophyceae indet.",
    "Syracosphaera pulchra > 20" : "Syracosphaera pulchra",
    "Syracosphaera pulchra > 20um" : "Syracosphaera pulchra",
    "Torodinium robustum > 20" : "Torodinium robustum", 
    "Cianophyceae indet." : "Cyanophyceae indet.", 
    "Coscinodiscophycea indet." : "Coscinodiscophycidae indet.", 
    "Flagellates indet. (ø inf. 20µm)" : "Protozoa indet. (ø inf. 20µm)", 
    "Pseudonitzschia fraudolenta" : "Pseudonitzschia fraudulenta", 
    "Pseudonitzschia delicatissima." : "Pseudonitzschia delicatissima", 
    "A. cf. aculeata" : "Acanthocardia sp."
    }

In [ ]:
taxa = np.array([match_taxa[taxon] if taxon in match_taxa.keys() else taxon for taxon in taxa])

In [ ]:
phyto_abund_raw["Taxon"] = taxa

In [ ]:
phyto_abund_raw[["Region", "id", "Longitude", "Latitude", "Closest_coast", "Day", "time", "SampleDepth", "Group", "Taxon", "Num_cell_l"]].to_csv("/mnt/d/PHD/MSFD/Data/Modulo1/phyto_abund_corrected.csv")
del phyto_abund_raw

## modify dataset 

subsituting in Marche samples with "<120" cell/L with uniform random number 

In [7]:
phyto_abundances = pd.read_csv(_HOME_ + "/PHD/MSFD/Data/Modulo1/phyto_abund_corrected.csv",index_col=0)

/tmp/ipykernel_2316/2115254911.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  phyto_abundances = pd.read_csv(_HOME_ + "/PHD/MSFD/Data/Modulo1/phyto_abund_corrected.csv",index_col=0)


In [8]:
phyto_abundances.loc[phyto_abundances["Num_cell_l"] == "<120", "Num_cell_l"] = np.vectorize(str)(np.random.randint(10, 120, phyto_abundances.loc[phyto_abundances["Num_cell_l"] == "<120", "Num_cell_l"].size))
phyto_abundances["Num_cell_l"] = phyto_abundances["Num_cell_l"].astype(float)
phyto_abundances.rename(columns={"SampleDepth" : "Sample_depth", "Day" : "Date"}, inplace=True)
phyto_abundances["Date"] = pd.to_datetime(phyto_abundances["Date"], format = '%d/%m/%Y')
phyto_abundances.drop_duplicates(inplace=True)
phyto_abundances = phyto_abundances[phyto_abundances["Num_cell_l"] != 0]

moving double sample to previous or following month when they are temporally close  

In [9]:
cond = (phyto_abundances["Region"] == "Emilia-Romagna") & ((phyto_abundances["Date"] == "2015-10-05") | (phyto_abundances["Date"] == "2015-10-09"))
phyto_abundances.loc[cond, "Date"] = datetime.datetime(2015, 9, 30)
cond = (phyto_abundances["Region"] == "Marche") & ((phyto_abundances["Date"] == "2016-03-08") | (phyto_abundances["Date"] == "2016-03-02"))
phyto_abundances.loc[cond, "Date"] = datetime.datetime(2016, 2, 27)
cond = (phyto_abundances["Region"] == "Marche") & (phyto_abundances["Date"] == "2017-04-04")
phyto_abundances.loc[cond, "Date"] = datetime.datetime(2017, 3, 30)
cond = (phyto_abundances["Region"] == "Marche") & (phyto_abundances["Date"] == "2017-05-02")
phyto_abundances.loc[cond, "Date"] = datetime.datetime(2017, 4, 30)
cond = (phyto_abundances["Region"] == "Veneto") & (phyto_abundances["Date"] == "2016-05-05")
phyto_abundances.loc[cond, "Date"] = datetime.datetime(2016, 4, 30)

grouping together higher cat. taxa 

In [10]:
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("sp\.|cf.\s.*", "spp.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Bacillariophyceae.*", "Diatomee indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Bacillariophyta.*", "Diatomee indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Chlorodendrophyceae.*", "Chlorophyta indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Chlorophyceae.*", "Chlorophyta indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Chroococcales indet.", "Cyanophyceae indet.", regex = False)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Chrysophyceae.*", "Chrysophyceae indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Coccolithophyceae.*", "Coccolithophyceae indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Dinophyceae indet. (ø inf. 20µm)", "Dinoflagellata indet.", regex = False)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Dinophyceae indet. (ø sup. 20µm)", "Dinoflagellata indet.", regex = False)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Noctilucea indet.", "Dinoflagellata indet.", regex = False)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Euglenophyceae.*", "Euglenophyceae indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Gymnodiniaceae.*|Gymnodiniales.*", "Gymnodiniales indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Naviculaceae indet.", "Diatomee indet.", regex = False)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Oscillatoriales indet.", "Cyanophyceae indet.", regex = False)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Pedinellales indet.", "Cyanophyceae indet.", regex = False)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Peridiniales.*", "Peridiniales indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("\(.*\)", "", regex = True, case=False)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Cyanophyceae indet. filamenti", "Cyanophyceae indet.", regex = False, case=False)

removing species var. cf. f.

In [11]:
taxons = phyto_abundances.loc[phyto_abundances["Taxon"].str.contains("var.", regex=False), "Taxon"].unique()
condition = phyto_abundances["Taxon"].isin(taxons)
phyto_abundances.loc[condition, "Taxon"] = phyto_abundances.loc[condition, "Taxon"].str.replace("var.*", "",regex=True)

In [12]:
condition = phyto_abundances["Taxon"].str.contains("cf\.|f\.", case=False, regex=True)
phyto_abundances.loc[condition, "Taxon"] = phyto_abundances.loc[condition, "Taxon"].str.replace("Cf\.| f\. .*", "", regex=True)

In [13]:
phyto_abundances = phyto_abundances.groupby(["Region", "id", "Longitude", "Latitude", "Closest_coast", "Date", "time", "Sample_depth", "Group", "Taxon"]).sum().reset_index()

In [14]:
condition = np.where(phyto_abundances["Taxon"].apply(lambda x: len(x.split())) == 1)[0]

Correcting other errors in the taxon column

In [11]:
phyto_abundances.loc[phyto_abundances["Taxon"] == "Rhoicosphenia", "Taxon"] == "Rhoicosphenia spp."
phyto_abundances.loc[phyto_abundances["Taxon"] == "Navicula", "Taxon"] == "Navicula spp."
phyto_abundances.loc[phyto_abundances["Taxon"] == "Protoperidinium", "Taxon"] = "Protoperidinium spp."
phyto_abundances.loc[phyto_abundances["Taxon"] == "Tetraselmis", "Taxon"] = "Tetraselmis spp."
phyto_abundances.loc[phyto_abundances["Taxon"] == " cylindrotheca", "Taxon"] = "Cylindrotheca spp."
phyto_abundances.loc[phyto_abundances["Taxon"] == "Kinetoplastidea", "Taxon"] = "Kinetoplastea indet."
phyto_abundances.loc[phyto_abundances["Taxon"] == "Gonyaulacaceae", "Taxon"] = "Gonyaulacaceae indet."
phyto_abundances.loc[phyto_abundances["Taxon"] == " tetraselmis", "Taxon"] = "Tetraselmis spp."
phyto_abundances.loc[phyto_abundances["Taxon"] == " phalacroma", "Taxon"] = "Phalacroma spp."
phyto_abundances.loc[phyto_abundances["Taxon"] == " leucocryptos", "Taxon"] = "Leucocryptos spp."
phyto_abundances.loc[phyto_abundances["Taxon"] == " centrodinium", "Taxon"] = "Centrodinium spp."
phyto_abundances.loc[phyto_abundances["Taxon"] == "Cyanobacteria", "Taxon"] = "Cyanobacteria indet."
phyto_abundances.loc[phyto_abundances["Taxon"] == " corymbellus aureus", "Taxon"] = "Corymbellus aureus"
phyto_abundances.loc[phyto_abundances["Taxon"] == "Mediophyceae", "Taxon"] = "Mediophyceae indet."
phyto_abundances.loc[phyto_abundances["Taxon"] == "Chaetoceros throndsenii ", "Taxon"] = "Chaetoceros throndsenii"

In [5]:
phyto_abundances =pd.read_csv(_HOME_ + "/PHD/MSFD/Data/Modulo1/phyto_abund_modified.csv")

In [12]:
phyto_abundances.to_csv(_HOME_ + "/PHD/MSFD/Data/Modulo1/phyto_abund_modified.csv")